In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os, getpass
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.callbacks import StdOutCallbackHandler
import gradio as gr
from agents import  RAGAgentLangChainOpenAI
from agents.rag_agents_lang_chain import rag_search_internal_documents

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/dk/8plcwpv53gg0_dg9fkx_rgr00000gn/T/ipykernel_85236/1046491335.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory=f"../db/rag_db", embedding_function=embedding)

In [5]:
llm_openai = ChatOpenAI(temperature=0.7 , model="gpt-4o-mini")

In [6]:
custom_prompt = ChatPromptTemplate.from_template(
"""Use the information below to answer the user's question.
If you can't find a suitable answer, just say you don't know.
Don't give wrong or incorrect answers.

{context}

Câu hỏi: {question}
"""
)

In [7]:
ragLangChainOpenAI = RAGAgentLangChainOpenAI(memory=memory , vector_store=vectorstore, prompt_template=custom_prompt, llm=llm_openai)

In [14]:
ragLangChainOpenAI.invoke("Who is Alex Chen")



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Alex Chen
Assistant: I don't know.
Follow Up Input: Who is Alex Chen
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Use the information below to answer the user's question.
If you can't find a suitable answer, just say you don't know.
Don't give wrong or incorrect answers.

# HR Record

# Alex Chen

## Summary
- **Date of Birth:** March 15, 1990  
- **Job Title:** Backend Software Engineer  
- **Location:** San Francisco, California  

## Insurellm Career Progression
- **April 2020:** Joined Insurellm as a Junior Backend Developer. Focused on building APIs to enhance customer data security.
- **October 

{'question': 'Who is Alex Chen',
 'chat_history': [HumanMessage(content='Alex Chen', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is Alex Chen', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Alex Chen is a Backend Software Engineer at Insurellm, located in San Francisco, California. He was born on March 15, 1990. Alex joined Insurellm in April 2020 as a Junior Backend Developer, was promoted to Backend Software Engineer in October 2021, and in March 2023, he was awarded the title of Senior Backend Software Engineer for his exemplary performance. He has a strong focus on building APIs and developing microservices architecture, significantly contributing to the company's backend solutions. Alex is also involved in diversity and inclusion initiatives and has plans for further professional development in AI and machine learning.", additional_kwargs={}, response_

In [12]:
def chat(question, history):
    result_conversation = ragLangChainOpenAI.invoke(question)
    return result_conversation["answer"]


view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [8]:
rag_internal_tools = rag_search_internal_documents(ragLangChainOpenAI)

In [9]:
rag_internal_tools.invoke("Who is Alex Chen")



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Use the information below to answer the user's question.
If you can't find a suitable answer, just say you don't know.
Don't give wrong or incorrect answers.

## Other HR Notes
- Participates regularly in Insurellm's Diversity & Inclusion initiatives, championing tech accessibility for underrepresented communities.
- Completed several certifications in cloud architecture and DevOps, contributing to professional growth.
- Plans for a professional development course in AI and machine learning to further enhance backend capabilities in Insurellm's offerings.
- Acknowledged for volunteer efforts in local tech meetups, bringing seasoned engineers to mentor aspiring coders.  

Alex Chen continues to be a vital asset at Insurellm, contributing significantly to innovative backend solutions that help shape the future of insurance

'Alex Chen is a Backend Software Engineer at Insurellm, based in San Francisco, California. He joined the company in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021. In March 2023, he became a Senior Backend Software Engineer due to his exemplary performance in scaling backend services. Alex is recognized for his contributions to innovative backend solutions, participation in Diversity & Inclusion initiatives, and ongoing professional development in cloud architecture and DevOps.'

In [10]:
import agents

prompt_templates = (
    """You are a Large Language Model trained to support users.
- Keep your answers SHORT and to the point.
- If you are unsure or do not have enough information, answer honestly: “I don’t know.”
- Absolutely do NOT make up stories or over-speculate.
"""
)


aiAgent = agents.AgentsOpenAIWithTools(tools=[rag_internal_tools],thread_id='thread_12', prompt_templates=prompt_templates)

In [12]:
print(aiAgent.invoke("search in internal document, Who is Alex Chen").to_dict())



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Who is Alex Chen
Assistant: Alex Chen is a Backend Software Engineer at Insurellm, based in San Francisco, California. He joined the company in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021. In March 2023, he became a Senior Backend Software Engineer due to his exemplary performance in scaling backend services. Alex is recognized for his contributions to innovative backend solutions, participation in Diversity & Inclusion initiatives, and ongoing professional development in cloud architecture and DevOps.
Follow Up Input: Alex Chen
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
P